### Product Performance: Measure product demand, pricing, and sales velocity.

### 1. Display average standard price for every product finish

In [ ]:
SELECT ProductFinish, AVG(ProductStandardPrice)
FROM Customer_T
GROUP BY ProductFinish; 

In [ ]:
Explaination: 

### 2. For all ordered products, list product IDs, their description and the number of times each product has been ordered.

In [ ]:
SELECT OLT.ProductID, ProductDescription, COUNT(OLT.OrderID) AS TimesOrdered
FROM OrderLine_T OLT, Product_T PT 
WHERE OLT.ProductID = PT.ProductID
GROUP BY OLT.ProductID, ProductDescription; 

### 3. List product ID, description and price of products with less than average standard price of  all products.

In [ ]:
SELECT ProductID, ProductDescription, ProductStandardPrice
FROM Product_T 
WHERE ProductStandardPrice < (
    SELECT AVG(ProductStandardPrice)
    FROM Product_T
    )
;

### 4. List product ID, description and price for products with less than average standard price that have been ordered more than once. Sort the result based on the product price. (Use subqueries)

In [ ]:
SELECT ProductID, ProductDescription, ProductStandardPrice
FROM Product_T PT
WHERE ProductStandardPrice < (
    SELECT AVG(ProductStandardPrice)
    FROM Product_T
    )
    AND PT.ProductID IN (
        SELECT OLT.ProductID
        FROM OrderLine_T OLT
        GROUP BY OLT.ProductID
        HAVING COUNt(*) > 1
    )
ORDER BY ProductStandardPrice;
/*These table references dont have to match as long as the keys are the same
(PT. Vs. OLT ==> ProductID)
Then, our last subquery's GROUP BY allows for us to use a Boolean in HAVING clause*/

### 5. Display the ProductFinish of products that appear in 3 or more orders

In [ ]:
SELECT DISTINCT ProductFinish
FROM Product_T 
WHERE ProductID IN (
	SELECT ProductID
	FROM OrderLine_T
	GROUP BY ProductID
	HAVING COUNT(*) >= 3
	)
;

### 6. Using a joing, Show description and total quantity of products that have been ordered a total of more than 3 quantity and are more expensive than the average price of product lines 1 and 2. 

In [ ]:
SELECT ProductDescription,  
SUM(OrderedQuantity) AS "How Many Sold"
FROM OrderLine_T OLT INNER JOIN Product_T PT
ON OLT.ProductID = PT.ProductID
WHERE EXISTS (
	SELECT *
	FROM Product_T
	WHERE ProductID = PT.ProductID
	AND ProductStandardPrice > (
		SELECT AVG(ProductStandardPrice)
		FROM Product_T 
		WHERE ProductLineID IN (1,2)
		)
	)
GROUP BY ProductDescription
HAVING "How Many Sold" > 3;

### 7. What quantity of each product finish has been ordered by customers  whose postal code starts with 9 or ends with 4? Show each Product finish along with its total quantity ordered.

In [ ]:
SELECT ProductFinish, SUM(OrderedQuantity) AS TotalQuantity
FROM Customer_T CT
INNER JOIN Order_T OT ON CT.CustomerID = OT.CustomerID
INNER JOIN OrderLine_T OLT ON OT.OrderID = OLT.OrderID
INNER JOIN Product_T PT ON OLT.ProductID = PT.ProductID
WHERE (CustomerPostalCode LIKE '9%' OR CustomerPostalCode LIKE '%4')
--Alt: ... LIKE "9%" OR ... LIKE "%4". i.e. No "(" or ")", but 9% & %4 in double quotes
GROUP BY ProductFinish;